# 속성 추출
1. 차트 진입후 5위 이내까지 진입한 일수
2. 해당 곡 아티스트의 다른 곡이 차트에 진입한 일수
3. 차트 진입 후 5위 이내까지 일간 랭킹 상승 평균치
4. and more...

In [1]:
from getAttribute import dbQuery, chartInDays, rankIncreaseMean, rankInDays

# 신경망 코드

In [2]:
import tensorflow as tf
import numpy as np

## Train dataset 추출

In [3]:
geniedf = dbQuery("SELECT * FROM zuzak.genie where YYMMDD = 180105 and ranking <= 15")
bugsdf = dbQuery("SELECT * FROM zuzak.bugs where YYMMDD = 180710 and ranking <= 15")
mnetdf = dbQuery("SELECT * FROM zuzak.mnet where YYMMDD = 180701 and ranking <= 15")

# 데이터프레임 합치기
df = geniedf.append(bugsdf)
df = df.append(mnetdf)
df.reset_index(inplace=True, drop=True)

### 레이블 설정

In [4]:
# 조작음원 = 1
# 여기선 지나오다, way back home 조작으로 가정
tmp = df[(df['title']=='WayBackHome') | (df['title']=='지나오다') | (df['title']=='그날처럼')]
labels = np.zeros([45, 1])
labels[tmp.index] = 1

### 속성 추출 및 정규화

In [6]:
# 이전에 차트에 든 일수
chartindays = np.fromiter(map(chartInDays, df['title'], df['artist'], ['genie']*len(df)), np.int32)

In [7]:
# 랭킹 상승 평균값
rankimean = np.fromiter(map(rankIncreaseMean, df['title'], ['genie']*len(df)), np.int32)
rankimean = np.array((rankimean+1) / (chartindays+1))

In [8]:
# top5 진입 일수
rankindays = np.fromiter(map(rankInDays, df['title'], ['genie']*len(df)), np.int32)
rankindays = np.array((rankindays + 1) / (chartindays+1))

In [9]:
# 데이터 정규하
chartindays = np.array((chartindays+0.01) / (chartindays+1))
chartindays

array([0.01      , 0.9988475 , 0.505     , 0.01      , 0.99938547,
       0.99876712, 0.01      , 0.01      , 0.99907649, 0.505     ,
       0.98862069, 0.505     , 0.99936498, 0.97893617, 0.99932099,
       0.99953236, 0.01      , 0.99927843, 0.99543779, 0.99877778,
       0.99927843, 0.01      , 0.99973243, 0.99707965, 0.99810707,
       0.99922535, 0.99545872, 0.99888136, 0.01      , 0.01      ,
       0.01      , 0.99927843, 0.99922535, 0.99810707, 0.98848837,
       0.99927843, 0.99973243, 0.99706231, 0.01      , 0.99933691,
       0.01      , 0.99440678, 0.99326531, 0.01      , 0.01      ])

### 속성 합치기

In [10]:
train_data = np.array([chartindays, rankimean, rankindays]).T
train_data

array([[1.00000000e-02, 1.00000000e+00, 3.50000000e+01],
       [9.98847497e-01, 1.11757858e-01, 2.32828871e-03],
       [5.05000000e-01, 1.50000000e+00, 2.25000000e+01],
       [1.00000000e-02, 3.00000000e+00, 3.30000000e+01],
       [9.99385475e-01, 6.20732464e-02, 1.24146493e-03],
       [9.98767123e-01, 1.74346202e-02, 3.73599004e-03],
       [1.00000000e-02, 1.00000000e+00, 5.40000000e+01],
       [1.00000000e-02, 2.00000000e+00, 6.80000000e+01],
       [9.99076493e-01, 9.04850746e-02, 1.86567164e-03],
       [5.05000000e-01, 1.00000000e+00, 2.25000000e+01],
       [9.88620690e-01, 1.14942529e+00, 2.29885057e-02],
       [5.05000000e-01, 5.00000000e+01, 1.00000000e+00],
       [9.99364978e-01, 6.41436818e-02, 1.28287364e-03],
       [9.78936170e-01, 2.08510638e+00, 4.25531915e-02],
       [9.99320988e-01, 6.79012346e-02, 1.37174211e-03],
       [9.99532357e-01, 4.67642891e-02, 9.44733113e-04],
       [1.00000000e-02, 1.40000000e+01, 8.00000000e+00],
       [9.99278426e-01, 7.28862

# 신경망 학습 

In [90]:
tf.reset_default_graph()
# [chartindays, rankimean, rankindays]
x_data = train_data
y_data = labels

#########
# 신경망 모델 구성
######
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W = tf.Variable(tf.random_uniform([3, 1], -1., 1.))

L = tf.nn.sigmoid(tf.matmul(X, W))

model = L

cost = tf.reduce_mean(tf.square(model-Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
train_op = optimizer.minimize(cost)

# 신경망 모델 학습
                      
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())
                      
ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())
    for step in range(100):
        sess.run(train_op, feed_dict={X: x_data, Y: y_data})
        print(step + 1, sess.run(cost, feed_dict={X: x_data, Y: y_data}))    
    saver.save(sess, './model/dnn.ckpt')

prediction = model
target = Y
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

1 0.42983988
2 0.42647746
3 0.42315543
4 0.41987428
5 0.4166349
6 0.41343787
7 0.4102837
8 0.4071729
9 0.40410596
10 0.40108317
11 0.3981048
12 0.39517102
13 0.3922821
14 0.38943803
15 0.3866389
16 0.3838847
17 0.38117537
18 0.37851062
19 0.37589043
20 0.37331453
21 0.37078264
22 0.36829436
23 0.36584938
24 0.36344722
25 0.36108738
26 0.35876927
27 0.3564924
28 0.354256
29 0.35205936
30 0.34990162
31 0.34778175
32 0.34569874
33 0.34365126
34 0.34163788
35 0.33965668
36 0.3377055
37 0.33578157
38 0.33388126
39 0.33199963
40 0.33013004
41 0.32826257
42 0.32638177
43 0.32446194
44 0.32245556
45 0.32026243
46 0.31762522
47 0.3136305
48 0.30171788
49 0.24473827
50 0.24343486
51 0.24219273
52 0.24099767
53 0.23984191
54 0.23872048
55 0.23762988
56 0.23656751
57 0.23553126
58 0.2345193
59 0.2335301
60 0.23256236
61 0.23161487
62 0.23068655
63 0.22977643
64 0.22888361
65 0.2280073
66 0.22714669
67 0.22630106
68 0.2254697
69 0.22465198
70 0.22384723
71 0.22305475
72 0.22227398
73 0.22150423
74 

# 테스트 데이터

In [98]:
df = dbQuery("SELECT * FROM zuzak.mnet where ranking <=15 and YYMMDD = 180710")

### 데이터 정규화

In [99]:
# 이전에 차트에 든 일수
chartindays = np.fromiter(map(chartInDays, df['title'], df['artist'], ['bugs']*len(df)), np.int32)
# 랭킹 상승 평균값
rankimean = np.fromiter(map(rankIncreaseMean, df['title'], ['bugs']*len(df)), np.int32)
rankimean = np.array((rankimean+0.01) / (chartindays+1))
# top5 진입 일수
rankindays = np.fromiter(map(rankInDays, df['title'], ['bugs']*len(df)), np.int32)
rankindays = np.array((rankindays + 0.01) / (chartindays+1))
# 정규화
chartindays = np.array((chartindays+0.01) / (chartindays+1))
# 데이터 합치기
test_data = np.array([chartindays, rankimean, rankindays]).T

### 테스트

In [100]:
test_data

array([[ 9.99476744e-01,  5.23308668e-02,  5.33826638e-04],
       [ 9.96055777e-01,  3.94462151e-01,  4.02390438e-03],
       [ 9.99152397e-01,  8.47688356e-02,  8.64726027e-04],
       [ 1.00000000e-02, -9.90000000e-01,  3.20100000e+01],
       [ 9.98760951e-01,  1.23917397e-01,  1.26408010e-03],
       [ 9.94406780e-01,  5.53728814e-01,  5.70621469e-03],
       [ 9.99087558e-01,  9.12534562e-02,  9.30875576e-04],
       [ 9.99152397e-01,  8.39126712e-02,  8.64726027e-04],
       [ 9.97991886e-01,  1.94746450e-01,  2.04868154e-03],
       [ 9.99725838e-01,  2.74189975e-02,  2.79700914e-04],
       [ 1.00000000e-02,  9.90100000e+01,  1.01000000e+00],
       [ 9.85000000e-01,  1.51515152e-04,  1.51515152e-04],
       [ 1.00000000e-02,  9.90100000e+01,  1.01000000e+00],
       [ 9.99202899e-01,  7.97181965e-02,  8.13204509e-04],
       [ 1.00000000e-02,  9.70100000e+01,  1.01000000e+00]])

In [101]:
x_data = test_data

X = tf.placeholder(tf.float32)
# 신경망에 가중치 W과 편향 b을 적용합니다

L = tf.nn.sigmoid(tf.matmul(X, W))

model = L

prediction = model
result = sess.run(prediction, feed_dict={X: x_data})
result = result.astype(np.float32)

print('예측값:', result)

예측값: [[0.26703808]
 [0.26496264]
 [0.26684088]
 [0.78779614]
 [0.26660296]
 [0.26401156]
 [0.26680145]
 [0.26684782]
 [0.2661854 ]
 [0.26718962]
 [0.01655629]
 [0.27032867]
 [0.01655629]
 [0.26687157]
 [0.01796539]]


In [102]:
df['prediction'] = result
df

,YYMMDD,ranking,title,artist,prediction
0,180710,1,DanceTheNightAway,TWICE(트와이스),0.267038
1,180710,2,1도없어,Apink(에이핑크),0.264963
2,180710,3,뚜두뚜두(DDU-DUDDU-DU),BLACKPINK,0.266841
3,180710,4,Dejavu,뉴이스트W,0.787796
4,180710,5,동화,멜로망스(Melomance),0.266603
5,180710,6,BAAM,모모랜드(MOMOLAND),0.264012
6,180710,7,장마(RainySeason),마마무(Mamamoo),0.266801
7,180710,8,ForeverYoung,BLACKPINK,0.266848
8,180710,9,너없인안된다,비투비,0.266185
9,180710,10,여행,볼빨간사춘기,0.267190
